# Mask-RCNN Model Inference in Amazon SageMaker

This notebook is a step-by-step tutorial on [Mask R-CNN](https://arxiv.org/abs/1703.06870) model inference using [Amazon SageMaker model deployment hosting service](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

To get started, we initialize an Amazon execution role and initialize a `boto3` session to find our AWS region name.

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

session = boto3.session.Session()
aws_region = session.region_name
print(f'AWS region:{aws_region}')

## Build and Push Amazon SageMaker Model Serving Images

For this step, the [IAM Role](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles.html) attached to this notebook instance needs full access to Amazon ECR service. If you created this notebook instance using the ```./stack-sm.sh``` script in this repository, the IAM Role attached to this notebook instance is already setup with full access to ECR service. 

Below, we have a choice of two different models for doing inference:

1. [TensorPack Faster-RCNN/Mask-RCNN](https://github.com/tensorpack/tensorpack/tree/master/examples/FasterRCNN)

2. [AWS Samples Mask R-CNN](https://github.com/aws-samples/mask-rcnn-tensorflow)

It is recommended that you build and push both Amazon SageMaker model <b>serving</b> container images below and use either image for serving the model for inference later.


### TensorPack Faster-RCNN/Mask-RCNN Model Serving Image

Use ```./container-serving/build_tools/build_and_push.sh``` script to build and push the TensorPack Faster-RCNN/Mask-RCNN <b>serving</b> image to Amazon ECR. 

In [ ]:
!cat ./container-serving/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container-serving/build_tools/build_and_push.sh {aws_region}

Set ```tensorpack_image``` below to Amazon ECR URI of the <b>serving</b> image you pushed above.

In [ ]:
tensorpack_image = '807253771232.dkr.ecr.us-east-1.amazonaws.com/mask-rcnn-tensorpack-serving-sagemaker:tf1.13-tp26664c3' #<amazon-ecr-uri>

### AWS Samples Mask R-CNN Model Serving Image
Use ```./container-serving-optimized/build_tools/build_and_push.sh``` script to build and push the AWS Samples Mask R-CNN <b>serving</b> image to Amazon ECR.

In [ ]:
!cat ./container-serving-optimized/build_tools/build_and_push.sh

Using your *AWS region* as argument, run the cell below.

In [ ]:
%%time
! ./container-serving-optimized/build_tools/build_and_push.sh {aws_region}

 Set ```aws_samples_image``` below to Amazon ECR URI of the <b>serving</b> image you pushed above.

In [ ]:
aws_samples_image = '807253771232.dkr.ecr.us-east-1.amazonaws.com/mask-rcnn-tensorflow-serving-sagemaker:tf1.13-153442b' #<amazon-ecr-uri>

## Amazon SageMaker Initialization 
We have built and pushed the <b>serving</b> images to Amazon ECR. Now we are ready to start using Amazon SageMaker for deploying model endpoint for inference. Next, we create a SageMaker sessiom.

In [ ]:
sagemaker_session = sagemaker.session.Session(boto_session=session)

Next, we set ```serving_image``` to either the `tensorpack_image` or the `aws_samples_image` variable you defined above.

In [ ]:
serving_image =  tensorpack_image # set to tensorpack_image or aws_samples_image 
print(f'serving image: {serving_image}')

## Define Amazon SageMaker Model
Next, we define Amazon SageMaker model that we will use to host the model on an Amazon SageMaker endpoint. The `model_url` below must point to a trained model. Please esnure that the trained model is consistent with the model serving image used for deploying the model.

In [ ]:
model_url = 's3://aws-ajayvohra-ml-data/mask-rcnn/sagemaker/output/mask-rcnn-s3-2020-02-06-18-12-32-906/output/model.tar.gz'
serving_container = {
    'Image': serving_image,
    'ModelDataUrl': model_url,
    'Mode': 'SingleModel',
    'Environment': { 'SM_MODEL_DIR' : '/opt/ml/model' }
}

create_model_response = sagemaker_session.create_model(name="mask-rcnn-model", 
                                                       role=role, 
                                                       container_defs=serving_container)

print(create_model_response)

Next, we define the Amazon SageMaker model hosted service endpoint configuration. Below we use a 1 instance of `ml.p3.2xlarge` (1 GPU) instance type to serve the model. 

In [ ]:
epc = sagemaker_session.create_endpoint_config(
    name="mask-rcnn-model-endpoint-config", 
    model_name="mask-rcnn-model", 
    initial_instance_count=1, 
    instance_type='ml.p3.2xlarge')
print(epc)

Next we specify the Amazon SageMaker endpoint name for the endpoint used to serve the model.

In [ ]:
endpoint_name='mask-rcnn-model-endpoint'

Next, we create the Amazon SageMaker endpoint using the endpoint configuration we created above.

In [ ]:
ep=sagemaker_session.create_endpoint(endpoint_name=endpoint_name, config_name=epc, wait=True)

Now that the Amazon SageMaker endpoint is in service, we will use the endpoint to do inference for test images. Below we download an example image from the web and use it as input for inference.

In [ ]:
import cv2
import requests
import os
import urllib

url="https://i.kinja-img.com/gawker-media/image/upload/c_scale,f_auto,fl_progressive,q_80,w_1600/ekpvcpwo560egadg2mvm.jpg"

img_id="image1.jpg"
img_local_path=os.path.join("/tmp", img_id)
if os.path.isfile(img_local_path):
    print(f"Removing existing tmp file:{img_local_path}")
    os.remove(img_local_path)
    
with urllib.request.urlopen(url) as url_stream:
    with open(img_local_path, 'wb') as f:
        f.write(url_stream.read())

img=cv2.imread(img_local_path, cv2.IMREAD_COLOR)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

Below we show the image we downloaded above.

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))

Next we invoke the Amazon SageMaker endpoint to detect objects in the image using the model we deployed to the endpoint above.

In [ ]:
import boto3 
import base64
import json

client = boto3.client('sagemaker-runtime')

with open(img_local_path, "rb") as image_file:
    img_data = base64.b64encode(image_file.read())
    data = {"img_id": img_id}
    data["img_data"] = img_data.decode('utf-8')
    body=json.dumps(data).encode('utf-8')
    
response = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=body)
body=response['Body'].read()
msg=body.decode('utf-8')
data=json.loads(msg)

The response from the endpoint includes annotations for the detected objects in COCO annotations format. 

Below we define a function to convert COCO run length encoding for image mask to a binary image mask.

In [ ]:
import numpy as np

def rle_to_binary_mask(rle, img_shape):
    value = 0
    mask_array = []
    for count in rle:
        mask_array.extend([int(value)]*count)
        value = (value + 1) % 2
    
    assert len(mask_array) == img_shape[0]*img_shape[1]
    b_mask = np.array(mask_array, dtype=np.uint8).reshape(img_shape)
    
    return b_mask

Below we define a function for generating random colors for visualizing annotations.

In [ ]:
import colorsys
import random

def random_colors(N, bright=False):
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

Below we define a function to apply a binary mask for an annotation to the image.

In [ ]:
def apply_binary_mask(image, mask, color, alpha=0.5):
    a_mask = np.stack([mask]*3, axis=2).astype(np.int8)
    for c in range(3):
        image[:, :, c] = np.where(mask == 1, image[:, :, c] *(1 - alpha) + alpha * color[c]*255,image[:, :, c])
    return image

Below we process all the images and apply the masks in the annotations. Each annotation also includes bounding box, cateory id  and cateogry class name, beside the segmentation mask.

In [ ]:
annotations = data['annotations']
colors = random_colors(len(annotations))
for i, a in enumerate(annotations):
    segm = a['segmentation']
    img_shape = tuple(segm['size'])
    rle = segm['counts']
    b_mask = rle_to_binary_mask(rle, img_shape)
    img = apply_binary_mask(img, b_mask, colors[i] )

Below we visualize the image with the applied segmentation masks.

In [ ]:
import matplotlib.pyplot as plt

fig,ax = plt.subplots(figsize=(img.shape[1]//50, img.shape[0]//50))
ax.imshow(img.astype(int))